In [ ]:
ls *.h5

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact
from matplotlib import colors
from glob import glob
from scipy import interp, interpolate
from scipy.interpolate import BSpline, CubicSpline
from tqdm import tqdm, trange

paths = sorted(glob('template_scan*.h5'))
H = {}
for path in paths:
    with h5py.File(path) as f:
        dset = f['adc_count_histo']
        H[path] = dset[...]
        extent = dset.attrs['extent']

        x_bin_edges = np.linspace(*extent[:2], dset.shape[1]+1)
        y_bin_edges = np.linspace(*extent[2:], dset.shape[2]+1)

        x_bin_center = (x_bin_edges[1:] + x_bin_edges[:-1]) / 2
        y_bin_center = (y_bin_edges[1:] + y_bin_edges[:-1]) / 2


    
@interact
def plot(pid=(0,1295)):
    N = len(H)
    fig, ax = plt.subplots(N+1, figsize=(12, 12), sharex=True)
    splines = []
    for i, (path, h) in enumerate(H.items()):
        _h = h[pid]
        
        N = _h.sum(axis=-1)
        mode = y_bin_center[_h.argmax(axis=-1)]
        mean = (_h * y_bin_center[None, :]).sum(axis=-1)  / N
        
        foo = (y_bin_center[None, :] - mean[:, None])**2
        std = np.sqrt((_h * foo).sum(axis=-1) / (N-1))
        average_std = np.nanmean(std)
        # good knowledge        
        G = N >= (N.mean() * 0.1)  
        
        img = ax[i].imshow(
            _h.T,
            origin='bottom',
            extent=extent,
            norm=colors.LogNorm()
        )
        plt.colorbar(img, ax=ax[i])
        
        #ax[i].plot(x_bin_center[G], mean[pid][G], 'o', color='red')
        #ax[i].plot(x_bin_center[G], mode[pid][G], 'd', color='orange')
        ax[i].errorbar(
            x=x_bin_center[G], 
            y=mean[G], 
            yerr=std[G],
            fmt='.', 
            color='red'
        )
        
        
        _x = x_bin_center[G]
        _y = mean[G]
        spl = CubicSpline(_x, _y)
        splines.append((spl, average_std))
        __x = np.linspace(x_bin_center.min(), x_bin_center.max(), 1000)
        ax[i].plot(__x, spl(__x), '-', color='magenta', lw=1)
        
        
        ax[i].set_aspect('auto')
    
        ax[i].set_ylabel('normalized amplitude')
        ax[i].set_title('Path:{}'.format(path))
        ax[i].grid()
    for spl, average_std in splines:
        __x = np.linspace(x_bin_center.min(), x_bin_center.max(), 1000)
        ax[-1].plot(__x, spl(__x), '-', label='avg std: {:.2f}'.format(average_std))
    ax[-1].grid()
    ax[-1].legend()
    
        
    plt.suptitle('Pixel: {}'.format(pid))
    plt.xlabel('time around 50% max height [ns]')
None

In [ ]:
splines = []

for pid in trange(1296):
    sub_splines = {}
    for path, h in H.items():
        _h = h[pid]
        
        N = _h.sum(axis=-1)
        mode = y_bin_center[_h.argmax(axis=-1)]
        mean = (_h * y_bin_center[None, :]).sum(axis=-1)  / N
        foo = (y_bin_center[None, :] - mean[:, None])**2
        std = np.sqrt((_h * foo).sum(axis=-1) / (N-1))
        
        average_std = np.nanmean(std)
        
        # good knowledge        
        G = N >= (N.mean() * 0.1)  
        _x = x_bin_center[G]
        _y = mean[G]
        spl = CubicSpline(_x, _y)

        __x = np.linspace(0, 20, 50)
        max_amplitude = spl(__x).max()
        sub_splines[(max_amplitude, average_std)] = spl
    
    keys = list(sub_splines.keys())
    average_stds = np.array([k[-1] for k in keys])
    max_amplitudes = np.array([k[0] for k in keys])
    if (average_stds < 0.05).all():
        splines.append(sub_splines[keys[np.argmax(max_amplitudes)]])
    else:
        splines.append(sub_splines[keys[np.argmin(average_stds)]])

In [ ]:
plt.figure(figsize=(12, 12))
for spl in splines:
    plt.plot(x_bin_center, spl(x_bin_center), 'k-', lw=1, alpha=0.1)
plt.grid()

In [ ]:
x = []
y = []
_x = np.linspace(x_bin_center.min(), x_bin_center.max(), 1000)

for spl in splines:
    x.append(_x)
    y.append(spl(_x))
    
x = np.concatenate(x)
y = np.concatenate(y)
plt.figure(figsize=(18, 12))
plt.hist2d(
    x, 
    y, 
    bins=(501, 501), 
    range=[extent[:2], extent[2:]],
    norm=colors.LogNorm()
)
plt.grid()
plt.colorbar()
None